# Scraping basics for Playwright

If you feel comfortable with scraping in general, you're free to skip this notebook and try to go right to the next one. Same thing if you get bored partway down.

> The [scraping section](https://jonathansoma.com/everything/scraping/) on my Everything I Know site might be helpful.
>
> I know I love them, but **you don't have to use CSS selectors!**

## Part 0: Imports

Import what you need to use Playwright, and start up a new browser to use for scraping. 

> If you end up opening a lot of Chromes/Chromiums, shutting down the Python kernel with the stop button is an easy way to make them go away! You'll have to re-run your notebook, but at least you won't have sixty icons in your dock.

In [1]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

## Part 1: Scraping by class

Scrape the content at http://jonathansoma.com/lede/static/by-class.html, printing out the title, subhead, and byline.

In [2]:
await page.goto("http://jonathansoma.com/lede/static/by-class.html")

<Response url='https://jonathansoma.com/lede/static/by-class.html' request=<Request url='https://jonathansoma.com/lede/static/by-class.html' method='GET'>>

In [3]:
await page.locator(".title").text_content()

'How to Scrape Things'

In [4]:
await page.locator(".subhead").text_content()

'Some Supplemental Materials'

In [5]:
await page.locator(".byline").text_content()

'By Jonathan Soma'

## Part 2: Scraping using tags

Scrape the content at http://jonathansoma.com/lede/static/by-tag.html, printing out the title, subhead, and byline.

In [6]:
await page.goto("http://jonathansoma.com/lede/static/by-tag.html")

<Response url='https://jonathansoma.com/lede/static/by-tag.html' request=<Request url='https://jonathansoma.com/lede/static/by-tag.html' method='GET'>>

In [7]:
await page.locator("h1").text_content()

'How to Scrape Things'

In [8]:
await page.locator("h3").text_content()

'Some Supplemental Materials'

In [9]:
await page.locator("p").text_content()

'By Jonathan Soma'

## Part 3: Scraping using a single tag

Scrape the content at http://jonathansoma.com/lede/static/by-list.html, creating a dictionary out of the title, subhead, and byline in sentences, e.g. "the title is `______`"

> **This will be important for the next few:** you can use `.get_by_text` but it seems kind of silly since maybe the text would change. I think getting them all, then using list indexes like `[0]`, etc, would be better! If I sold you on CSS selectors, you can also look up `nth-of-type` and use it with `.select_one`.

In [10]:
await page.goto("http://jonathansoma.com/lede/static/by-list.html")

<Response url='https://jonathansoma.com/lede/static/by-list.html' request=<Request url='https://jonathansoma.com/lede/static/by-list.html' method='GET'>>

In [11]:
await page.locator("p").nth(0).text_content()

'How to Scrape Things'

In [12]:
await page.locator("p").nth(1).text_content()

'Some Supplemental Materials'

In [13]:
await page.locator("p").nth(2).text_content()

'By Jonathan Soma'

## Part 4: Scraping a single table row

Scrape the content at http://jonathansoma.com/lede/static/single-table-row.html, printing out the title, subhead, and byline in sentences, e.g. "the title is `______`."

In [14]:
await page.goto("http://jonathansoma.com/lede/static/single-table-row.html")

<Response url='https://jonathansoma.com/lede/static/single-table-row.html' request=<Request url='https://jonathansoma.com/lede/static/single-table-row.html' method='GET'>>

In [15]:
title = await page.locator('td').nth(0).text_content()

print(f'The title is {title}.')

The title is How to Scrape Things.


In [16]:
subhead = await page.locator("td").nth(1).text_content()

print(f'The subhead is {subhead}.')

The subhead is Some Supplemental Materials.


In [17]:
byline = await page.locator("td").nth(2).text_content()

print(f'The byline is {byline}.')

The byline is By Jonathan Soma.


## Part 5: Saving into a dictionary

Scrape the content at http://jonathansoma.com/lede/static/single-table-row.html, saving the title, subhead, and byline into a single dictionary called `book`.

> Don't use pandas for this one!

In [33]:
await page.goto("http://jonathansoma.com/lede/static/single-table-row.html")

<Response url='https://jonathansoma.com/lede/static/single-table-row.html' request=<Request url='https://jonathansoma.com/lede/static/single-table-row.html' method='GET'>>

In [34]:
import requests
from bs4 import BeautifulSoup

In [35]:
html = await page.content()
doc = BeautifulSoup(html)

In [36]:
cells= doc.select("td")
cells

[<td>How to Scrape Things</td>,
 <td>Some Supplemental Materials</td>,
 <td>By Jonathan Soma</td>]

In [37]:
book = {
    'title': cells[0].text,
    'subhead': cells[1].text,
    'byline':cells[2].text
}
book

{'title': 'How to Scrape Things',
 'subhead': 'Some Supplemental Materials',
 'byline': 'By Jonathan Soma'}

## Part 6: Scraping multiple table rows

Scrape the content at http://jonathansoma.com/lede/static/multiple-table-rows.html, printing out each title, subhead, and byline.

> You won't use pandas for this one, either!

In [38]:
await page.goto("http://jonathansoma.com/lede/static/multiple-table-rows.html")

<Response url='https://jonathansoma.com/lede/static/multiple-table-rows.html' request=<Request url='https://jonathansoma.com/lede/static/multiple-table-rows.html' method='GET'>>

In [39]:
html = await page.content()
doc = BeautifulSoup(html)

In [40]:
rows = doc.select("tr")
rows

[<tr>
 <td>How to Scrape Things</td>
 <td>Some Supplemental Materials</td>
 <td>By Jonathan Soma</td>
 </tr>,
 <tr>
 <td>How to Scrape Many Things</td>
 <td>But, Is It Even Possible?</td>
 <td>By Sonathan Joma</td>
 </tr>,
 <tr>
 <td>The End of Scraping</td>
 <td>Let's All Use CSV Files</td>
 <td>By Amos Nathanos</td>
 </tr>]

In [45]:
for row in rows:
    print("-----")
    cells=row.select("td")
    print(cells[0].text)
    print(cells[1].text)
    print(cells[2].text)

-----
How to Scrape Things
Some Supplemental Materials
By Jonathan Soma
-----
How to Scrape Many Things
But, Is It Even Possible?
By Sonathan Joma
-----
The End of Scraping
Let's All Use CSV Files
By Amos Nathanos


## Part 7: Scraping an actual table

Scrape the content at http://jonathansoma.com/lede/static/the-actual-table.html, creating a list of dictionaries.

> Don't use pandas here, either, even though that's exactly what we did in class.

In [28]:
await page.goto("http://jonathansoma.com/lede/static/the-actual-table.html")

<Response url='https://jonathansoma.com/lede/static/the-actual-table.html' request=<Request url='https://jonathansoma.com/lede/static/the-actual-table.html' method='GET'>>

In [46]:
html = await page.content()
doc = BeautifulSoup(html)

In [48]:
all_books=[]

for row in rows:
    print("-----")
    cells=row.select("td")
    book = {
    'title': cells[0].text,
    'subhead': cells[1].text,
    'byline':cells[2].text
    }
    all_books.append(book)
all_books

-----
-----
-----


[{'title': 'How to Scrape Things',
  'subhead': 'Some Supplemental Materials',
  'byline': 'By Jonathan Soma'},
 {'title': 'How to Scrape Many Things',
  'subhead': 'But, Is It Even Possible?',
  'byline': 'By Sonathan Joma'},
 {'title': 'The End of Scraping',
  'subhead': "Let's All Use CSV Files",
  'byline': 'By Amos Nathanos'}]

## Part 8: Scraping multiple table rows into a list of dictionaries

Scrape the content at http://jonathansoma.com/lede/static/the-actual-table.html, creating a pandas DataFrame.

> There are two ways to do this one! One uses just pandas, the other one uses the result from Part 7.

In [49]:
import pandas as pd
df = pd.DataFrame(all_books)
df

,title,subhead,byline
0,How to Scrape Things,Some Supplemental Materials,By Jonathan Soma
1,How to Scrape Many Things,"But, Is It Even Possible?",By Sonathan Joma
2,The End of Scraping,Let's All Use CSV Files,By Amos Nathanos


In [51]:
tables = pd.read_html("http://jonathansoma.com/lede/static/the-actual-table.html")
df = tables[0]
df

,0,1,2
0,How to Scrape Things,Some Supplemental Materials,By Jonathan Soma
1,How to Scrape Many Things,"But, Is It Even Possible?",By Sonathan Joma
2,The End of Scraping,Let's All Use CSV Files,By Amos Nathanos


## Part 9: Scraping into a file

Scrape the content at http://jonathansoma.com/lede/static/the-actual-table.html and save it as `output.csv`

In [52]:
df.to_csv("output.csv", index=False)